In [63]:
import json 
import pyRofex as pr  
import csv 
import yfinance as yf
import math 

In [64]:

tickers = yf.Tickers("ggal pamp.ba ypf")

In [65]:
# Símbolo del bono del Tesoro a 10 años

# Obtener datos de Yahoo Finance
treasury_yield = yf.Ticker("^TNX")
 
# Obtener datos históricos recientes
history = treasury_yield.history(period="1mo")
# Obtener la tasa de interés más reciente
latest_rate = (history['Close'][-1])/100


In [66]:

spot_ggal = (tickers.tickers['GGAL'].info)['currentPrice']
spot_pamp = (tickers.tickers['PAMP.BA'].info)['currentPrice']
spot_ypfd = (tickers.tickers['YPF'].info)['currentPrice']


In [67]:

# Set the the parameter for the REMARKET environment
pr.initialize(user="antoniopacosantos11540",
                   password="dggybB0#",
                   account="REM11540",
                   environment=pr.Environment.REMARKET)

In [84]:

instruments = [
    'GGAL/OCT24', 
    'GGAL/AGO24',
    'GGAL/AGO24/OCT24', 
    'PAMP/OCT24',
    'PAMP/AGO24',
    'PAMP/AGO24/OCT24',
    'YPFD/OCT24',
    'YPFD/AGO24',
    'YPFD/AGO24/OCT24',
    'DLR/AGO24'
]

spot_for_future = {
    'GGAL/OCT24': spot_ggal,
    'GGAL/AGO24': spot_ggal,
    'GGAL/AGO24/OCT24': spot_ggal,
    'PAMP/OCT24': spot_pamp,
    'PAMP/AGO24': spot_pamp,
    'PAMP/AGO24/OCT24': spot_pamp,
    'YPFD/OCT24': spot_ypfd,
    'YPFD/AGO24': spot_ypfd,
    'YPFD/AGO24/OCT24': spot_ypfd
}

In [69]:
from datetime import date

In [70]:
current_date = date.today()

In [71]:
def parse_maturity_date_and_compute_tValue(maturity_date): 
    maturity_parsed_date = date(int(maturity_date[0:4]), int(maturity_date[4:6]), int(maturity_date[6:8]))
    tValue = (maturity_parsed_date - current_date).days / 365
    return tValue    

In [72]:
def compute_lending_rate(price,spot, tValue):
    return math.log(price/spot) / tValue

def compute_borrowing_rate(price,spot, tValue):
    return math.log(price/spot) / tValue

In [73]:
entries = [
    pr.MarketDataEntry.BIDS,
    pr.MarketDataEntry.OFFERS,
]

lending_rates = {}
borrowing_rates = {}


In [74]:
###TESTING 

instruments = ['BTC/JUL24']
spot_for_future = {'BTC/JUL24': spot_ggal}

#all entries
entries = [
    pr.MarketDataEntry.BIDS,
    pr.MarketDataEntry.OFFERS,
    pr.MarketDataEntry.LAST,
    pr.MarketDataEntry.CLOSING_PRICE,
    pr.MarketDataEntry.OPENING_PRICE,
    pr.MarketDataEntry.HIGH_PRICE,
    pr.MarketDataEntry.LOW_PRICE,
    pr.MarketDataEntry.SETTLEMENT_PRICE,
    pr.MarketDataEntry.NOMINAL_VOLUME,
    pr.MarketDataEntry.TRADE_EFFECTIVE_VOLUME,
    pr.MarketDataEntry.TRADE_VOLUME,
    pr.MarketDataEntry.OPEN_INTEREST
]

In [88]:
pr.get_market_data(ticker=instruments[0],
                entries=entries
                )

{'status': 'OK',
 'marketData': {'OI': None,
  'CL': None,
  'EV': 0.0,
  'HI': None,
  'BI': [],
  'LA': None,
  'NV': 0,
  'SE': {'price': 4700.0, 'size': None, 'date': 1720137600000},
  'LO': None,
  'OF': [],
  'OP': None,
  'TV': 0},
 'depth': 1,
 'aggregated': True}

In [89]:
# 2-Defines the handlers that will process the messages and exceptions.
def market_data_handler(message):
    print("Market Data Message Received: {0}".format(message))
    #update lending rates if the bid is not None 
    bid = message['marketData']['BI']
    offer = message['marketData']['OF']            
    if bid is not None and bid != []:
        symbol = message['instrumentId']['symbol']
        bid_price = bid[0]['price']
        spot = spot_for_future[symbol]
        tValue = parse_maturity_date_and_compute_tValue(pr.get_instrument_details(symbol)['instrument']['maturityDate'])
        lending_rates[symbol] = compute_borrowing_rate(spot, bid_price, tValue)
        if compute_lending_rate(spot, bid_price, tValue) < latest_rate:
            print(f"La tasa de interés implícita colocadora de {symbol} es menor a la tasa de interés del bono del Tesoro a 10 años más reciente. Oportunidad de arbitraje")
    #update borrowing rates if the offer is not None
    if offer is not None and offer != []: 
        symbol = message['instrumentId']['symbol']
        offer_price = offer[0]['price']
        spot = spot_for_future[symbol]
        tValue = parse_maturity_date_and_compute_tValue(pr.get_instrument_details(symbol)['instrument']['maturityDate'])
        borrowing_rates[symbol] = compute_lending_rate(spot, offer_price, tValue)
        if compute_lending_rate(spot, offer_price, tValue) > latest_rate:
            print(f"La tasa de interés implícita tomadora de {symbol} es mayor a la tasa de interés del bono del Tesoro a 10 años más reciente. Oportunidad de arbitraje")
        
def error_handler(message):
    print("Error Message Received: {0}".format(message))


def exception_handler(e):
    print("Exception Occurred: {0}".format(e))

In [77]:

pr.init_websocket_connection(market_data_handler=market_data_handler,
                                  error_handler=error_handler,
                                  exception_handler=exception_handler)
pr.market_data_subscription(tickers=instruments,entries=entries)

pr.order_report_subscription()
#time.sleep(5)
#pr.close_websocket_connection()


In [78]:
print(borrowing_rates)

{}


Market Data Message Received: {'type': 'Md', 'timestamp': 1720280138519, 'instrumentId': {'marketId': 'ROFX', 'symbol': 'BTC/JUL24'}, 'marketData': {'OI': None, 'CL': None, 'EV': 0.0, 'HI': None, 'BI': [], 'LA': None, 'NV': 0, 'SE': {'price': 84860000.0, 'date': 1720137600000}, 'LO': None, 'OF': [], 'OP': None, 'TV': 0}}
